<a href="https://colab.research.google.com/github/hiris25/Tierpsy-Tracking-Analysis/blob/main/50th_simple_ttest_heatmap_plot_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Two Variable Tracking Feature Analysis Pipeline

## **Read in and prepare data**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy as scipy 
from scipy import stats
from scipy.stats import ttest_ind

%config InlineBackend.figure_format = 'retina'

read in file containing all genotypes, with the first coloumn the genotype, second column n number for that genotype, other coloums are features

**Here you need to change the directory of your source .csv file.**


In [ ]:
df = pd.read_csv('filelocation/filename.csv')

###For single worm tracking keep only 50th percentile and remove derivatives

We only really need to consider the 50th percentile value as these are single worms not multiple worms (ie. each row is one worm)
So at this point maybe we can make a new df with only 50th percenticle, no derivatives and add back genotype and any other variables.

To add additional coloums back to the new df, unhash the line

df2['variable'] = df['variable']

and replace 'varible' with the name of your coloumn

In [ ]:
df2 = df.filter(regex='_50th', axis=1)

df2['genotype'] = df['genotype']

unwanted = df2.columns[df2.columns.str.startswith('d_')]

df3 = df2.drop(unwanted, axis=1)

unwanted2 = df3.columns[df3.columns.str.contains('_std_')]

df50 = df3.drop(unwanted2, axis=1)

df50.head()

##Calculate significantly different features

Here we are doing multiple t-tests for two groups which are defined as group1 and group2. 

**Change 'N2' and 'genotype' to the genotypes or conditions you want to compare before running**

*For each new analysis in the same dataset run a new ttest*

In [ ]:
# get a list of all columns in the dataframe without the Group column
column_list = [x for x in df50.columns if x != 'genotype'] #tyrmaine yes no??
# create an empty dictionary
t_test_results = {}
# loop over column_list and execute code explained above
for column in column_list:
    group1 = df50.where(df50.genotype== 'nameofstrain1').dropna(how='all')[column]
    group2 = df50.where(df50.genotype== 'nameofstrain2').dropna(how='all')[column]
    # add the output to the dictionary 
    t_test_results[column] = scipy.stats.ttest_ind(group1,group2)
results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
results_df.columns = ['t-statistic','pvalue']

This tells you the number of features that were analysed, if this number is much lower than 592 then there might be a lot of missing data

In [ ]:
results_df.shape

(542, 2)

This tells you the number of significant features p<0.05

In [ ]:
dfsg = results_df[results_df.pvalue <= 0.05]

dfsg.shape

(10, 2)

This sorts the values by pvalue and shows you the top 25
if you want to see more change 25 to whichever number of top features you'd like to see

In [ ]:
dfsort = dfsg.sort_values(by=['pvalue'])

dfsort.head(25)

###Export data into csv and save file

Export significant results into a new .csv file

**Change the name of the file to match the analysis before running** 

If you want to export all the P value results then change 'dfsort' to 'results_df'

In [ ]:
results_df.to_csv(r'filelocation/filename.csv')

##Generate heatmap of all analysed genotypes

Here we will make a heatmap that indicates p-value vs N2 for each feature and each analysed genotype. 

This can be useful if you want to show overall similarity/difference between your strains.

First we need to create a dataframe with all the features in rows and each coloumn represents p-value vs N2. 

Before this point you should have run all the analyses that you want and exported as .csv files

**Here you need to read in all the anlyses that you've carried out and change the coloumn name to the correct genotype**

In [ ]:
df1 = pd.read_csv('filelocation/filename.csv', index_col = 0)
df1 = df1.rename(columns={df1.columns[1]: "nameofstrain1"})

df2 = pd.read_csv('filelocation/filename.csv', index_col = 0)
df2 = df2.rename(columns={df2.columns[1]: "nameofstrain2"})

df3 = pd.read_csv('filelocation/filename.csv', index_col = 0)
df3 = df3.rename(columns={df3.columns[1]: "nameofstrain3"})

df4 = pd.read_csv('filelocation/filename.csv', index_col = 0)
df4 = df4.rename(columns={df4.columns[1]: "nameofstrain4"})

df5 = pd.read_csv('filelocation/filename.csv', index_col = 0)
df5 = df5.rename(columns={df5.columns[1]: "nameofstrain5"})


In [ ]:
dfpvalues = pd.concat([df1, df2, df3, df4, df5], axis=1)

unwanted3 = dfpvalues.columns[dfpvalues.columns.str.contains('t')]

dfpvalues = dfpvalues.drop(unwanted3, axis=1)

dfpvalues.head()

Generating heatmap

In [ ]:
cmap = sns.cubehelix_palette(8, reverse=True, as_cmap=True)

plt.figure(figsize=(10,15))
ax = sns.heatmap(dfpvalues, yticklabels=10, vmin=0, vmax=0.05, cmap=cmap)

##Plot interesting features

Get a list of the significant features which can be used to query the original dataset and return the raw values for plotting.

Here  you should also add back the genotype and other varible coloumns.

In [ ]:
features = dfsort.index.values

listfeat = list(features)

listfeat.append('genotype')

listfeat

Using the list created above we can select our features of interest from the whole dataset, **which can then be exported as a .csv if the line is unhashed.**

In [ ]:
dfintersting = df50[listfeat]

#dfintersting.to_csv(r'filelocation/filename.csv')

dfintersting.head()

Here we can plot a simple violin plot of one feature across all the genotypes.

**Change feature name to the feature of interest**

Delete hue="genotype" if you do not want a legened

In [ ]:
ax = sns.violinplot(x='genotype', y="feature", hue="genotype", data=df50, palette="colorblind")

In [ ]:
cx = sns.barplot(x="genotype", y="feature", hue="genotype", data=df50, palette="colorblind", ci=68)